<a href="https://colab.research.google.com/github/rituraj123450/AI-Project-Web-Application-for-object-identification-/blob/main/UPL_Assignment_203190027.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from FeatureExtractor import FeatureExtractor
from BOW import BOW
from pybrain.datasets.supervised import SupervisedDataSet
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
import cv2
import numpy as np
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class LoadImage:
    def __init__(self):
        pass

    def loadTrainImage(self,path):
        '''
        load all train images from the path
        :param path: the images' path
        :return: the type and the name of all insects, the path of all insect images
        '''

        # get all species' names
        insectSpecies = os.listdir(path)

        trainingPaths = []
        names = []
        # get full list of all training images
        for p in insectSpecies:
            paths = os.listdir(path + "\\" + p)
            for j in paths:
                trainingPaths.append(path + "\\" + p + "\\" + j)
                names.append(p)

        return insectSpecies,names,trainingPaths

In [ ]:
class FeatureExtractor:
    def __init__(self):
        pass

    def getSiftFeature(self, image):
        '''
        get the sift features from a insect image
        :param image:
        :return:the descriptor of the image
        '''
        sift = cv2.SIFT()
        imgBlur = cv2.GaussianBlur(image, (5, 5), 0)  # Remove noise
        gray = cv2.cvtColor(imgBlur, cv2.CV_LOAD_IMAGE_GRAYSCALE)
        kp, dsc = sift.detectAndCompute(gray, None)  # get keypoint and descriptor
        return dsc

    def getSurfFeature(self, image):
        '''
        get the surf features from a insect image
        :param image:
        :return: the descriptor of the image
        '''
        surf = cv2.SURF()
        imgBlur = cv2.GaussianBlur(image, (5, 5), 0)  # Remove noise
        gray = cv2.cvtColor(imgBlur, cv2.CV_LOAD_IMAGE_GRAYSCALE)
        kp, dsc = surf.detectAndCompute(gray, None)  # get keypoint and descriptor
        return dsc


    def getSingleFeature(self, path, bowDictionary, featureExtraType):
        '''
        get an image's feature by using Bag of words dictionary
        :param path:image's path
        :param bowDictionary:the dictionary of Bag of words
        :param featureExtraType:the feature type:sift or surf
        :return:
        '''
        im = cv2.imread(path, 1)
        gray = cv2.cvtColor(im, cv2.CV_LOAD_IMAGE_GRAYSCALE)
        if (featureExtraType.upper() == "SIFT"):
            return bowDictionary.compute(gray, cv2.SIFT().detect(gray))
        if (featureExtraType.upper() == "SURF"):
            return bowDictionary.compute(gray, cv2.SURF().detect(gray))

In [ ]:
class BOW:
    def __init__(self):
        pass

    def getBowDictionary(self,dictionarySize, descriptors, featureExtraType):
        '''
        get the dictionary of Bog of words
        :param dictionarySize: the size of dictionary
        :param descriptors: the all images' descriptors
        :param featureExtraType: the feature type: sift or surf
        :return: the dictionary of Bag of words
        '''
        BOW = cv2.BOWKMeansTrainer(dictionarySize)
        for dsc in descriptors:
            BOW.add(dsc)

        # dictionary created
        dictionary = BOW.cluster()

        if (featureExtraType.upper() == "SIFT"):
            extra = cv2.DescriptorExtractor_create("SIFT")
        if (featureExtraType.upper() == "SURF"):
            extra = cv2.DescriptorExtractor_create("SURF")
        bowDictionary = cv2.BOWImgDescriptorExtractor(extra, cv2.BFMatcher(cv2.NORM_L2))
        bowDictionary.setVocabulary(dictionary)
        return bowDictionary

    def getDescriptors(self, path, featureExtraType):
        '''
        get all descriptors from images in the path
        :param path: the image's path
        :param featureExtraType: the feature type:sift or surf
        :return: all images' descriptors
        '''
        featureExtra = FeatureExtractor()
        descriptors = []
        for p in path:
            image = cv2.imread(p)
            if (featureExtraType.upper() == "SIFT"):
                dsc = featureExtra.getSiftFeature(image)
            if (featureExtraType.upper() == "SURF"):
                dsc = featureExtra.getSurfFeature(image)
            descriptors.append(dsc)

        return descriptors

In [ ]:

class Classify:
    def __init__(self):
        pass


    def algoSVM(self,trainingPath, testPath, featureExtraType):
        '''
        Using Support Vector Machine algorithm to classify insect images
        :param trainingPath: the path of training images
        :param testPath: he path of testing images
        :param featureExtraType: the feature type : sift or surf
        :return:
        '''
        loadImage = LoadImage()
        featureExtra = FeatureExtractor()
        bow = BOW()

        #get the species,the name of all insects, the path of all insect images
        insectSpecies, names, trainingPaths = loadImage.loadTrainImage(trainingPath)
        print(insectSpecies)
        print("Length of Data :", len(insectSpecies))
        dictionarySize = len(insectSpecies)

        insect = {}
        num = 1
        for name in insectSpecies:
            insect[name] = num
            num += 1

        #get the descriptors of all training images
        descriptors = bow.getDescriptors(trainingPaths, featureExtraType)
        #get Bag of Words dictionary
        bowDictionary = bow.getBowDictionary(dictionarySize, descriptors, featureExtraType)
        print("bow dictionary")

        #train data
        trainDesc = []
        #train response
        trainLabels = []
        i = 0

        #initialize train datas and train responses
        for p in trainingPaths:
            trainDesc.extend(featureExtra.getSingleFeature(p, bowDictionary, featureExtraType))
            trainLabels.append(insect[names[i]])
            i = i + 1

        svm = cv2.SVM()
        #training
        svm.train(np.array(trainDesc), np.array(trainLabels))

        testInsectNames = os.listdir(testPath)
        # Initialize a zero matrix to save the classification results
        result = np.zeros((dictionarySize, dictionarySize))
        print("result zero")

        count = 0
        #classify all the test immages
        for test in testInsectNames:
            testingImage = os.listdir(testPath + "\\" + test)
            for p in testingImage:
                #get feature from a test image
                feature = featureExtra.getSingleFeature(testPath + "\\" + test + "\\" + p, bowDictionary, featureExtraType)
                #predict
                p = svm.predict(feature)
                #save the result in the result matrix
                result[count, p - 1] += 1

            count += 1

        return result

    def algoANN(self, trainingPath, testPath, featureExtraType, epochs):
        '''
        Using Artificial Neural Network algorithm to classify insect images
        :param trainingPath: the path of training images
        :param testPath:  the path of testing images
        :param featureExtraType:  the feature type:sift or surf
        :param epochs: the numbre of training for neural network
        :return: the classification results
        '''
        loadImage = LoadImage()
        featureExtra = FeatureExtractor()
        bow = BOW()

        # get the species,the name of all insects, the path of all insect images
        insectNames, names, trainingPaths = loadImage.loadTrainImage(trainingPath)
        insectSpecies = len(insectNames)
        print("insect species:", insectSpecies)

        #get all descriptos of training images
        trainDescriptors = bow.getDescriptors(trainingPaths, featureExtraType)
        #get the BoW dictionary of trianing images
        trainBowDictionary = bow.getBowDictionary(insectSpecies, trainDescriptors, featureExtraType)

        #initialize a Neural Network
        net = buildNetwork(insectSpecies, 100, 100, insectSpecies)
        #initialize a data set
        ds = SupervisedDataSet(insectSpecies, insectSpecies)
        species = 0
        #add all datas in data set
        for p in insectNames:
            trainingPaths = os.listdir(trainingPath + "\\" + p)
            for j in trainingPaths:
                #add data
                ds.addSample(featureExtra.getSingleFeature(trainingPath + "\\" + p + "\\" + j,
                                                           trainBowDictionary, featureExtraType)[0], (species,))
            species += 1

        #initialize a trainer
        trainer = BackpropTrainer(net, ds, learningrate=0.01, momentum=0.1, weightdecay=0.01)
        #training
        for i in range(1, epochs):
            traError = trainer.train()
            print('after %d epochs,train error:%f' % (i, traError))

        testInsectNames, testNames, testingPaths = loadImage.loadTrainImage(testPath)
        testDescriptors = bow.getDescriptors(testingPaths, featureExtraType)
        testBowDictionary = bow.getBowDictionary(insectSpecies, testDescriptors, featureExtraType)
        # Initializes a zero matrix to save the classification results
        result = np.zeros((insectSpecies, insectSpecies))

        count = 0
        # classify all the test immages
        for m in testInsectNames:
            testPaths = os.listdir(testPath + "\\" + m)
            for n in testPaths:
                test = net.activate(featureExtra.getSingleFeature(testPath + "\\" + m + "\\" + n,
                                                                  testBowDictionary, featureExtraType)[0])
                target = map(lambda x: (x), test)  # numpy.array to list
                result[count, target.index(max(target))] += 1
            count += 1

        return result

In [ ]:
if __name__ == "__main__":
    start = time.clock()
    classify = Classify()
    #please change the file path accordingly
    trainingPath = "C:\\Users\\ace\\Desktop\\newData\\training"
    testPath = "C:\\Users\\ace\\Desktop\\newData\\testing"

    featureExtraType = "surf"
    result = classify.algoSVM(trainingPath, testPath, featureExtraType)

    end  = time.clock()
    print("run time: %f s" % (end - start))

    print(result)
    print("Error:%f" % (result.trace() / result.sum()))